In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import nltk

In [ ]:
spam =pd.read_csv("../data/spam.csv")

In [ ]:
# You might have seen this error message before; 
# there are symbols in the data file that are invalid 
# unicode characters.  Apparently this file isn't
# in unicode. 

In [ ]:
spam =pd.read_csv("../data/spam.csv", encoding="latin-1")

In [ ]:
spam.head()

In [ ]:
spam.iloc[:,3].value_counts()

In [ ]:
spam.iloc[0]

In [ ]:
# Ok, I don't know what those last three columns are for, but I don't like them.
spamclean = []
for line in open("spam.csv", encoding="latin-1"):
        a = line.find(",")
        spamclean.append((line[0:a], line[a+1:]))

In [ ]:
spam = pd.DataFrame(spamclean)

In [ ]:
spam = spam.drop(labels=0, axis=0)  #only do this once

In [ ]:
spam

In [ ]:
spam =spam.rename({0: "label", 1:"text"}, axis=1)

In [ ]:
# Create two dataframes.. 
s1 = spam.query("label == 'ham'")
s2 = spam.query("label == 'spam'")

In [ ]:
# Create two giant strings... 
s1text = s1.text.str.cat()
s2text = s2.text.str.cat()

In [ ]:
# tokenize the strings (proably split on whitespace)
tok1 = nltk.word_tokenize(s1text)
tok2 = nltk.word_tokenize(s2text)

In [ ]:
# And create frequency distribution objects.
d1 = nltk.FreqDist(tok1)
d2 = nltk.FreqDist(tok2)

In [ ]:
d1  # This will act like a dictionary.

In [ ]:
# These methods look like number of token counts
d1.N(), d2.N()  

In [ ]:
# and these look like the number of distinct tokens.
len(d1.keys()), len(d2.keys()) 

In [ ]:
# Ten thousand words.  This is reasonable.


In [ ]:
teststring = "Hey, what's up rich?"
teststring2 = "Do you want free airtime?"

In [ ]:
# here is the frequency data I need.
# Word, ham_occur, ham_total, spam_occur, spam_total
print("Word\tham_occ\tham_tot\tspam_occ\tspam_tot")

for token in nltk.word_tokenize(teststring):
    print(token, '\t', end="" )
    print(d1[token] , '\t',  d1.N(),'\t', d2[token], 
          '\t', d2.N())

In [ ]:
testset = ["Hey there, I am Maya with GP Research. We're surveying IL residents. Can you respond to a few questions?",
           "Headed down now.",
"The banana chocolate bread is delicious! All that’s left is one small heel, which I will dunk in my coffee tomorrow.",
"Hurry! For a limited time, add a FREE line to your account. Really, it's on us–no strings attached." ]


In [ ]:
# I'm going to reorganize the data from two dictionaries into a dataframe 
# so I can make scatterplots.
counts = pd.DataFrame()
for k in set(d1.keys()).union(set(d2.keys())):
    counts = counts.append( { "word": k , "d1": d1[k], "d2": d2[k]}, ignore_index=True)

In [ ]:
# We can make the plot more expressive by putting labels on the dots, since each
# dot is a word.  Because of flaws in matplotlib, we have to use ugly log10 axes here;
# plotting labels on points does not play nicely with log-log axes.

fig, ax = plt.subplots(figsize=(10,10))
ranks = counts.sort_values("d1", ascending=False).reset_index()

ax.scatter(np.log10(ranks.index), np.log10(ranks.d1+0.1))
ax.set_xlabel("log10 word rank")
ax.set_ylabel("log10 word count")

ax.grid()
for i in np.power(10, np.linspace(0,4,100)).astype("int"):
        ax.text(np.log10(ranks.index[i])+0.2, np.log10(ranks.loc[i,"d1"]+.1), ranks.loc[i,"word"],  fontsize=7)


In [ ]:
len(counts)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.loglog(counts.d1/d1.N(), counts.d2/d2.N(),'.')
ax.plot([1E-4,1E-1], [1E-4,1E-1], color='k', linestyle='-',linewidth=0.5)
ax.set_xlabel('Ham frequencies (log scale)')
ax.set_ylabel('Spam frequencies (log scale)')
for i in counts.index:
    if counts.loc[i, "d1"] + counts.loc[i, "d2"] >=50 :
        ax.annotate( counts.loc[i,"word"], ((counts.loc[i, "d1"]+.1)/d1.N(), (counts.loc[i,"d2"]+.1)/d2.N()), fontsize=8 )

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.loglog(counts.d1+counts.d2, counts.d1/d1.N()/counts.d2*d2.N(),'.')
#ax.plot([1,1E-1], [1,1E-1], color='k', linestyle='-',linewidth=0.5)
ax.set_xlabel('odds ratio in favor of ham')
ax.set_ylabel('number of occurrences total')
for i in counts.index:
    if counts.loc[i, "d1"] + counts.loc[i, "d2"] >=50 :
       ax.annotate( counts.loc[i,"word"], (counts.loc[i, "d1"]+counts.loc[i, "d2"], 
                (counts.loc[i, "d1"]+.1)/d1.N()/(counts.loc[i,"d2"]+.1)*d2.N()), fontsize=8 )